In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [3]:
def S1_GBM(S0: float, mu: list, sigma: list, T: float, n: int, nsim: int) -> list:
    '''
    S0: initial asset price
    mu: mean rate of asset return
    sigma: asset volatility
    T: time period
    n: number of steps
    nsim: number of simulation of the asset price process
    '''
    dt = T/n
    dw = np.sqrt(dt)*np.random.standard_normal(size=(nsim,n)) # generate the (nsim, n) brownian motion sample
    dz = (mu-0.5*sigma**2)*dt + sigma*dw 
    Sn = np.exp(np.cumsum(dz, axis=1)) # generate the (nsim, n) geometric brownian momtion price process    
    Sn = S0*np.concatenate((np.ones(shape=(nsim,1)),Sn), axis=1) # add the initial value

    return (Sn)

In [67]:
def BSprice(St: list, K: float, r: float, sigma: float, div: float, Tm: float, optype: str) -> float:

    d0 = sigma*np.sqrt(Tm)
    d1 = (np.log(St/K) + (r-div + 0.5*sigma**2)*Tm)/d0
    d2 = d1 - d0
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)

    if optype == 'call':
        BSprice = St*np.exp(-div*Tm)*Nd1 - np.exp(-r*Tm)*K*Nd2
    elif optype == 'put':
        BSprice = np.exp(-r*Tm)*K*(1-Nd2) - St*np.exp(-div*Tm)*(1-Nd1)
    else: raise Exception('optype must be call or put')

    return (BSprice)

def BSdelta(St: list, K: float, r: float, sigma: float, div: float, Tm: float, optype: str) -> float:
    d0 = sigma*np.sqrt(Tm)
    d1 = (np.log(St/K) + (r-div + 0.5*sigma**2)*Tm)/d0
    d2 = d1 - d0
    Nd1 = norm.cdf(d1)

    if optype == 'call':
        BSdelta = Nd1
    elif optype == 'put':
        BSdelta = -(1-Nd1)
    else: raise Exception('optype must be call or put')

    return(BSdelta)

